In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
from tensorflow.python.client import device_lib

In [2]:
f = open('../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()
word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

f = open("train.pickle", 'rb')
full_data = pickle.load(f)
f.close()

In [3]:
# Model Description
sense_word = 'hard'
model_name = 'basic'
model_dir = 'output/' + sense_word + '/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

In [4]:
# Parameters
mode = 'train'
num_senses = 3
batch_size = 64
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 100
keep_prob = 0.5
l2_lambda = 0.001
init_lr = 0.001
decay_steps = 5000
decay_rate = 0.96
clip_norm = 1
clipping = True

In [5]:
# MODEL
x = tf.placeholder('int32', [batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [batch_size], name="y")
x_mask  = tf.placeholder('bool', [batch_size, max_sent_size], name='x_mask') 
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 1)

with tf.name_scope("word_embedding"):
    if mode == 'train':
        unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))
    else:
        unk_word_emb_mat = tf.get_variable("word_emb_mat", shape=[unk_vocab_size, word_emb_size], dtype='float')
        
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)
    Wx = tf.nn.embedding_lookup(final_word_emb_mat, x)  

with tf.variable_scope("lstm"):
    cell_fw = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, input_keep_prob=input_keep_prob)
    d_cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, input_keep_prob=input_keep_prob)
    
    (fw_h, bw_h), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw, d_cell_bw, Wx, sequence_length=x_len, dtype='float', scope='lstm')
    h = tf.concat([fw_h, bw_h], 2)

def attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
    return c

with tf.variable_scope("attention"):
    W_att = tf.Variable(tf.truncated_normal([2*hidden_size, 1], -0.1, 0.1), name="W_att")
#     W_att = tf.get_variable("W_att", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(stddev=0.1))
    c = tf.expand_dims(attention(h[0], x_mask[0], W_att), 0)
    for i in range(1, batch_size):
        c = tf.concat([c, tf.expand_dims(attention(h[i], x_mask[i], W_att), 0)], 0)
        
with tf.variable_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([2*hidden_size, num_senses], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([num_senses]), name="b")
    drop_c = tf.nn.dropout(c, input_keep_prob)
    logits = tf.matmul(drop_c, W) + b
    predictions = tf.argmax(logits, 1)

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
global_step = tf.Variable(0, trainable=False, name="global_step")

learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

tv_all = tf.trainable_variables()
tv_regu =[]
for t in tv_all:
    if t.name.find('b:')==-1:
        tv_regu.append(t)
        
# l2 Loss
l2_loss = l2_lambda * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])

total_loss = loss + l2_loss

# Optimizer for loss
optimizer = tf.train.AdamOptimizer(learning_rate)

# Gradients and Variables for Loss
grads_vars = optimizer.compute_gradients(total_loss)

# Clipping of Gradients
clipped_grads = grads_vars
if(clipping == True):
    clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

# Training Optimizer for Total Loss
train_op = optimizer.apply_gradients(clipped_grads, global_step=global_step)

# Summaries
var_summaries = []
for v in tv_all:
    var_summary = tf.summary.histogram("{}/var".format(v.name), v)
    var_summaries.append(var_summary)

var_summaries_merged = tf.summary.merge(var_summaries)

loss_summary = tf.summary.scalar("loss", loss)
total_loss_summary = tf.summary.scalar("total_loss", total_loss)
summary = tf.summary.merge_all()

In [6]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
print (device_lib.list_local_devices())
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
saver = tf.train.Saver()                                             # For Saving the model
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 663743614070355765
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 7958282240
locality {
  bus_id: 1
}
incarnation: 7296158630257511041
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:06:00.0"
]


In [7]:
# Splitting
data_x = full_data[sense_word][0]
data_y = full_data[sense_word][2]
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, train_size=0.8, shuffle=True, stratify=data_y,random_state=0)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.9, shuffle=True, stratify=y_train,random_state=0)

/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [8]:
def data_prepare(x):
    num_examples = len(x)

    xx = np.zeros([num_examples, max_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_sent_size], dtype=bool)

    for j in range(num_examples):
        for i in range(max_sent_size):
            if(i>=len(x[j])):
                break
            w = x[j][i]
            xx[j][i] = word2id[w] if w in word2id else word2id['UNKNOWN_TOKEN']
            xx_mask[j][i] = True
            
    return xx, xx_mask

def eval_f1(yy, pred):
    num_batches = int(len(yy)/batch_size)
    f1 = f1_score(yy[:batch_size*num_batches], pred, average='macro')
    return f1*100

def model(xx, yy, mask, train_cond=True):
    num_batches = int(len(xx)/batch_size)
    losses = 0
    preds = []
    for j in range(num_batches): 
        
        s = j * batch_size
        e = (j+1) * batch_size
        
        feed_dict = {x:xx[s:e], y:yy[s:e], x_mask:mask[s:e], is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, total_loss, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)  
#             print("Steps:{}".format(step), ", Loss: {}".format(_loss))

        else:
            _loss, pred = sess.run([total_loss, predictions], feed_dict)
            preds.append(pred)
            
        losses +=_loss

    if(train_cond==False):
        y_pred = []
        for i in range(num_batches):
            for pred in preds[i]:
                y_pred.append(pred)
        return losses/num_batches, y_pred
    
    return losses/num_batches, step

In [9]:
x_id_train, mask_train = data_prepare(x_train)
x_id_val, mask_val = data_prepare(x_val)

In [ ]:
num_epochs = 10

for i in range(num_epochs):
    
#     random = np.random.choice(num_train_data, size=(num_train_data), replace=False)
#     X_train = train_x[random]
#     Y_train = train_y[random]
#     X_mask_train = train_x_mask[random]
            
    losses, step = model(x_id_train, y_train, mask_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    saver.save(sess, save_path=save_dir)                         
    print("Saved Model Complete")
    val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
    print("Validation F1 Score: ",  eval_f1(y_val, val_pred), "Loss: ", val_loss)

Epoch: 1 Step: 48 loss: 1.82926099251
Saved Model Complete
Validation F1 Score:  53.63321982 Loss:  1.17401063442
Epoch: 2 Step: 96 loss: 1.11383954932
Saved Model Complete
Validation F1 Score:  59.1958679247 Loss:  0.83121432066
Epoch: 3 Step: 144 loss: 0.883702294901
Saved Model Complete
Validation F1 Score:  55.1539201539 Loss:  0.746579027176
Epoch: 4 Step: 192 loss: 0.783861645808
Saved Model Complete
Validation F1 Score:  62.8137674428 Loss:  0.706567692757
Epoch: 5 Step: 240 loss: 0.728817914923
Saved Model Complete
Validation F1 Score:  61.4268755935 Loss:  0.691197013855
Epoch: 6 Step: 288 loss: 0.681750098243
Saved Model Complete
Validation F1 Score:  63.2787144672 Loss:  0.653878629208
Epoch: 7 Step: 336 loss: 0.652686534449
Saved Model Complete
Validation F1 Score:  61.7522459973 Loss:  0.601454377174
Epoch: 8 Step: 384 loss: 0.635066057245
Saved Model Complete
Validation F1 Score:  62.0320468613 Loss:  0.577779072523
Epoch: 9 Step: 432 loss: 0.626689401766


In [12]:
num_epochs = 10

for i in range(num_epochs):
    
#     random = np.random.choice(num_train_data, size=(num_train_data), replace=False)
#     X_train = train_x[random]
#     Y_train = train_y[random]
#     X_mask_train = train_x_mask[random]
            
    losses, step = model(x_id_train, y_train, mask_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    saver.save(sess, save_path=save_dir)                         
    print("Saved Model Complete")
    val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
    print("Validation F1 Score: ",  eval_f1(y_val, val_pred), "Loss: ", val_loss)

Epoch: 1 Step: 528 loss: 0.566895972316
Saved Model Complete
Validation F1 Score:  59.6627723644 Loss:  0.616566610336
Epoch: 2 Step: 576 loss: 0.541725418841
Saved Model Complete
Validation F1 Score:  57.2286782195 Loss:  0.583407258987
Epoch: 3 Step: 624 loss: 0.518973579009
Saved Model Complete
Validation F1 Score:  60.7465598686 Loss:  0.591347539425
Epoch: 4 Step: 672 loss: 0.502147963891
Saved Model Complete
Validation F1 Score:  61.2281249535 Loss:  0.559031885862
Epoch: 5 Step: 720 loss: 0.504496208703
Saved Model Complete
Validation F1 Score:  67.9733964027 Loss:  0.540830481052
Epoch: 6 Step: 768 loss: 0.489853277802
Saved Model Complete
Validation F1 Score:  58.599431969 Loss:  0.533681505919
Epoch: 7 Step: 816 loss: 0.472809040609
Saved Model Complete
Validation F1 Score:  71.6526593334 Loss:  0.450166010857
Epoch: 8 Step: 864 loss: 0.441281999772
Saved Model Complete
Validation F1 Score:  71.3799876156 Loss:  0.484917122126
Epoch: 9 Step: 912 loss: 0.452627406456
Saved Mod

In [13]:
num_epochs = 10

for i in range(num_epochs):
    
#     random = np.random.choice(num_train_data, size=(num_train_data), replace=False)
#     X_train = train_x[random]
#     Y_train = train_y[random]
#     X_mask_train = train_x_mask[random]
            
    losses, step = model(x_id_train, y_train, mask_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    saver.save(sess, save_path=save_dir)                         
    print("Saved Model Complete")
    val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
    print("Validation F1 Score: ",  eval_f1(y_val, val_pred), "Loss: ", val_loss)

Epoch: 1 Step: 1008 loss: 0.441487664357
Saved Model Complete
Validation F1 Score:  62.9939714641 Loss:  0.534826517105
Epoch: 2 Step: 1056 loss: 0.421977446104
Saved Model Complete
Validation F1 Score:  72.7674718834 Loss:  0.465573859215
Epoch: 3 Step: 1104 loss: 0.426068828752
Saved Model Complete
Validation F1 Score:  76.6903103319 Loss:  0.409250974655
Epoch: 4 Step: 1152 loss: 0.413043253124
Saved Model Complete
Validation F1 Score:  73.2880743562 Loss:  0.508180660009
Epoch: 5 Step: 1200 loss: 0.400123088621
Saved Model Complete
Validation F1 Score:  72.5467856615 Loss:  0.466546261311
Epoch: 6 Step: 1248 loss: 0.393948272491
Saved Model Complete
Validation F1 Score:  73.6301097277 Loss:  0.478032624722
Epoch: 7 Step: 1296 loss: 0.384558051825
Saved Model Complete
Validation F1 Score:  74.1522313564 Loss:  0.421771156788
Epoch: 8 Step: 1344 loss: 0.416114263237
Saved Model Complete
Validation F1 Score:  69.4316877153 Loss:  0.474960458279
Epoch: 9 Step: 1392 loss: 0.37518389523


In [14]:
num_epochs = 20

for i in range(num_epochs):
    
#     random = np.random.choice(num_train_data, size=(num_train_data), replace=False)
#     X_train = train_x[random]
#     Y_train = train_y[random]
#     X_mask_train = train_x_mask[random]
            
    losses, step = model(x_id_train, y_train, mask_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    saver.save(sess, save_path=save_dir)                         
    print("Saved Model Complete")
    val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
    print("Validation F1 Score: ",  eval_f1(y_val, val_pred), "Loss: ", val_loss)

Epoch: 1 Step: 1488 loss: 0.378261028789
Saved Model Complete
Validation F1 Score:  75.7727652464 Loss:  0.414021897316
Epoch: 2 Step: 1536 loss: 0.37483346307
Saved Model Complete
Validation F1 Score:  80.3922686946 Loss:  0.391434136033
Epoch: 3 Step: 1584 loss: 0.381940787037
Saved Model Complete
Validation F1 Score:  75.612394958 Loss:  0.425130355358
Epoch: 4 Step: 1632 loss: 0.366450667071
Saved Model Complete
Validation F1 Score:  71.1439100792 Loss:  0.421375405788
Epoch: 5 Step: 1680 loss: 0.383654830977
Saved Model Complete
Validation F1 Score:  78.2604064705 Loss:  0.419776910543
Epoch: 6 Step: 1728 loss: 0.381254092169
Saved Model Complete
Validation F1 Score:  73.2769438338 Loss:  0.435799384117
Epoch: 7 Step: 1776 loss: 0.361801016144
Saved Model Complete
Validation F1 Score:  75.2299932896 Loss:  0.417018574476
Epoch: 8 Step: 1824 loss: 0.387450824802
Saved Model Complete
Validation F1 Score:  77.9925413202 Loss:  0.466790676117
Epoch: 9 Step: 1872 loss: 0.369093801205
S

In [15]:
num_epochs = 30

for i in range(num_epochs):
    
#     random = np.random.choice(num_train_data, size=(num_train_data), replace=False)
#     X_train = train_x[random]
#     Y_train = train_y[random]
#     X_mask_train = train_x_mask[random]
            
    losses, step = model(x_id_train, y_train, mask_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    saver.save(sess, save_path=save_dir)                         
    print("Saved Model Complete")
    val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
    print("Validation F1 Score: ",  eval_f1(y_val, val_pred), "Loss: ", val_loss)

Epoch: 1 Step: 2448 loss: 0.314001502159
Saved Model Complete
Validation F1 Score:  84.9479731833 Loss:  0.358269560337
Epoch: 2 Step: 2496 loss: 0.319434905425
Saved Model Complete
Validation F1 Score:  80.1050572908 Loss:  0.405624371767
Epoch: 3 Step: 2544 loss: 0.311950338694
Saved Model Complete
Validation F1 Score:  83.0266353488 Loss:  0.423165607452
Epoch: 4 Step: 2592 loss: 0.327509464075
Saved Model Complete
Validation F1 Score:  82.9746219475 Loss:  0.362470671535
Epoch: 5 Step: 2640 loss: 0.320245316873
Saved Model Complete
Validation F1 Score:  84.2279290236 Loss:  0.377321755886
Epoch: 6 Step: 2688 loss: 0.306154819826
Saved Model Complete
Validation F1 Score:  80.2640436019 Loss:  0.365677559376
Epoch: 7 Step: 2736 loss: 0.325944423365
Saved Model Complete
Validation F1 Score:  80.6645489435 Loss:  0.374142676592
Epoch: 8 Step: 2784 loss: 0.31367650535
Saved Model Complete
Validation F1 Score:  79.6068543913 Loss:  0.394069182873
Epoch: 9 Step: 2832 loss: 0.33599482663
S

In [ ]:

saver.restore(sess, tf.train.latest_checkpoint(save_dir))
embed_mat = sess.run(embedding)